In [6]:
'''
超短线套利策略
每天收盘价委托买入，第二天成交后按盈利1%价委托卖出
重复以上
'''

In [ ]:
%matplotlib inline
import tushare as ts
import pymongo
import numpy as np
from datetime import datetime

from vnpy.trader.vtObject import VtBarData
from vnpy.trader.app.ctaStrategy.ctaBase import DAILY_DB_NAME
from vnpy.trader.app.ctaStrategy.ctaTemplate import CtaTemplate
from vnpy.trader.app.ctaStrategy.ctaBacktesting import BacktestingEngine

In [20]:
#合约代码
#code = '600036'
#capital = 1000000
exchange = 'SSE'
#fromDate = '20170101'

code = raw_input('code:')
fromDate = raw_input('date:')
capital = int(raw_input('capital:'))
#exchange = raw_input('exchange:')

vtCode = '.'.join([code, exchange])

code:600380
date:20170101
capital:1000000


In [146]:
'''
获取数据
'''
#下载数据
theDate = datetime.strptime(fromDate, '%Y%m%d')
data = ts.get_hist_data(code, theDate.strftime('%Y-%m-%d'))
data.sort_index(inplace = True)
print '数据下载完成'

#获取数据库
client = pymongo.MongoClient('localhost', 27017)
collection = client[DAILY_DB_NAME][vtCode]
collection.create_index('datetime')
print 'MongoDB连接成功'

for date, row in data.iterrows():
    bar = VtBarData()
    bar.open = row['open']
    bar.close = row['close']
    bar.high = row['high']
    bar.low = row['low']
    bar.volume = row['volume']
    
    bar.date = '%s' % date
    bar.datetime = datetime.strptime(date, '%Y-%m-%d')
    bar.exchange = exchange
    bar.vtSymbol = vtCode
    bar.symbol = code
    
    collection.update_many({'datetime':bar.datetime}, {'$set':bar.__dict__}, upsert = True)
print '数据更新完成'

数据下载完成
MongoDB连接成功
数据更新完成


In [42]:
'''
策略
'''
class ShortTermStratege(CtaTemplate):
    """双均线策略"""
    className = 'DoubleMaStrategy'
    author = 'loe'
    
    #策略变量
    capital = 0
    buyOrderId = ''
    sellOrderId = ''
    cost = 0
    startClose = 0
    endClose = 0
    
    #参数列表，保存了参数的名称
    paramList = ['name', 'className', 'author', 'vtSymbol']
    #变量列表，保存了变量的名称
    varList = ['inited', 'trading', 'pos']
    
    def __init__(self, ctaEngine, setting):
        super(ShortTermStratege, self).__init__(ctaEngine, setting)
    
    def onInit(self):
        '''启动策略（必须由用户继承实现）'''
        self.writeCtaLog(u'双均线策略初始化')
        self.putEvent()
    
    def onStart(self):
        '''初始化策略（必须由用户继承实现）'''
        self.writeCtaLog(u'双均线策略启动')
        self.putEvent()
    
    def onStop(self):
        '''停止策略（必须由用户继承实现）'''
        self.writeCtaLog(u'双均线策略停止')
        self.putEvent()
        
    def onTick(self, tike):
        '''收到行情Tick推送（必须由用户继承实现'''
        '''实盘时需要使用Tike数据在这里自行合成K线'''
        pass
    
    def onBar(self, bar):
        '''收到bar数据推送（必须由用户继承实现）'''
        if not self.startClose:
            self.startClose = bar.close
        self.endClose = bar.close
        
        if self.pos:
            #有持仓
            if not len(self.sellOrderId):
                #没有卖单委托，开卖单
                self.sellOrderId = self.sell(self.cost * 1.01, self.pos)
                
                #print bar.datetime, '卖出委托:%r' % (self.cost * 1.01)
        else:
            #有买单没有成交，取消买单委托
            if len(self.buyOrderId):
                self.cancelOrder(self.buyOrderId)
            #开新的买单
            #根据现有现金计算可买数量
            orderPrice = bar.close*1.05
            volume = int(self.capital / orderPrice)
            volume = volume - volume % 100
            self.buyOrderId = self.buy(orderPrice, volume)
            
            #print bar.datetime, '买入委托：%r' % orderPrice, '买入量：%r' % volume, '收盘价：%r' % bar.close
            
        
        self.putEvent()

    def onOrder(self, order):
        '''收到委托变化推送（必须由用户继承实现）'''
        '''对于无需做细粒度委托控制的策略，可以忽略onOrder'''
        pass
    
    def onTrade(self, trade):
        '''收到成交推送（必须由用户继承实现）'''
        if trade.direction == u'多':
            print 'symbol:%r' % trade.vtSymbol + 'direction:%s' % trade.direction
            #改变持有的现金
            self.capital -= trade.price * trade.volume
            #买单成交
            self.buyOrderId = ''
            #买入成交成本，用于计算下一次卖出的委托价格
            self.cost = trade.price
            
            #print trade.dt, '做多价格：%r' % trade.price, '成交量：%r' % trade.volume, '现金：%r' % self.capital, '\n'
        elif trade.direction == u'空':
            #改变持有的现金
            self.capital += trade.price * trade.volume
            #卖单成交
            self.sellOrderId = ''
            #买入成本清零
            self.cost = 0
            
            #print trade.dt, '做空价格：%r' % trade.price, '现金：%r' % self.capital, '\n'
    
    def onStopOrder(self, so):
        '''收到停止单推送（必须由用户继承实现）'''
        raise 'onStopOrder'
        pass

In [43]:
'''
回测
'''
#初始资金
ShortTermStratege.capital = capital

#创建回测引擎实例
engine = BacktestingEngine()

#设置引擎的回测模式为K线
engine.setBacktestingMode(engine.BAR_MODE)

#设置回测的起始日期
engine.setStartDate(fromDate, initDays = 0) #这里的initDays是真正起作用的，20天是指自然日肯定不够的

#设置产品相关参数
engine.setSlippage(0)           #滑点设置为0
engine.setRate(1.5/10000)       #股票交易手续费万1.5
engine.setSize(1)               #股票每股为1
engine.setPriceTick(0.01)      #股票最小价格变动
engine.setCapital(ShortTermStratege.capital) #初始资金为1

#设置使用的数据库
engine.setDatabase(DAILY_DB_NAME, vtCode)

#引擎中创建策略对象
engine.initStrategy(ShortTermStratege, {})

engine.strategy.vtSymbol = '健康元'

#开始跑回测
engine.runBacktesting()

#print '初始收盘价：%r' % engine.strategy.startClose, '结束收盘价：%r' % engine.strategy.endClose, '涨跌：%r' % (((engine.strategy.endClose / engine.strategy.startClose)-1) * 100), '\n' 

#显示回测结果
#engine.showDailyResult()

2017-10-19 10:25:11.274000	开始载入数据
2017-10-19 10:25:11.293000	载入完成，数据量：183
2017-10-19 10:25:11.293000	开始回测
2017-10-19 10:25:11.293000	策略初始化完成
2017-10-19 10:25:11.293000	策略启动完成
2017-10-19 10:25:11.293000	开始回放数据
symbol:''direction:多
symbol:''direction:多
symbol:''direction:多
symbol:''direction:多
symbol:''direction:多
symbol:''direction:多
symbol:''direction:多
2017-10-19 10:25:11.302000	数据回放结束


In [11]:
a = 'abc'
if a:
    print 'yes'
else:
    print 'no'

yes
